### Preprocessing

In [19]:
import sys 
import os
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from pathlib import Path

import cdt
cdt.SETTINGS.rpath = "/usr/bin/Rscript"
from cdt.causality.graph import PC

import pandas as pd
from src.features import preprocessing
import numpy as np
import networkx as nx
import time

In [10]:
data_path = Path('/home/ralmasri/projects/Thesis/Domain-Guided-Monitoring/data/')
csv_path = data_path / "logs_aggregated_concurrent.csv"
huawei_config = preprocessing.HuaweiPreprocessorConfig()
huawei_config.aggregated_log_file = csv_path
preprocessor = preprocessing.ConcurrentAggregatedLogsPreprocessor(huawei_config)
huawei_df = preprocessor._load_log_only_data().fillna("")

Generating DRAIN clusters from log_df: 100%|██████████| 1252/1252 [00:00<00:00, 3698.69it/s]


In [11]:
df_sizes = [100, 1000, 10000, 50000]
huawei_dfs = [huawei_df.head(x) for x in df_sizes]
results = {}

### Heuristic

In [15]:
from src.features import sequences
from collections import Counter

causality_preprocessor = preprocessing.ConcurrentAggregatedLogsCausalityPreprocessor(huawei_config)
huawei_config.min_causality = 0.01

def collect_sequence_metadata(grouped_data):
    transformer = sequences.load_sequence_transformer()
    return transformer.collect_metadata(grouped_data, "all_events")

def generate_heuristic_knowledge(huawei_df):
    log_only_data = huawei_df.copy()
    log_only_data["grouper"] = 1
    grouped_data = preprocessor._aggregate_per(log_only_data, aggregation_column="grouper")
    metadata = collect_sequence_metadata(grouped_data)

    relevant_columns = set(
        [
            x
            for x in preprocessor.relevant_columns
            if not causality_preprocessor.config.log_only_causality or "log" in x
        ]
    )
    counted_causality = causality_preprocessor._generate_counted_causality(
        huawei_df, relevant_columns
    )

    causality_records = []
    for from_value, to_values in counted_causality.items():
        total_to_counts = len(to_values)
        to_values_counter = Counter(to_values)
        for to_value, to_count in to_values_counter.items():
            if to_count / total_to_counts > causality_preprocessor.config.min_causality:
                causality_records.append(
                    {
                        "parent_id": from_value,
                        "parent_name": from_value.split("#")[1],
                        "child_id": to_value,
                        "child_name": to_value.split("#")[1],
                    },
                )

    return (
        pd.DataFrame.from_records(causality_records)
        .drop_duplicates()
        .reset_index(drop=True)
    )

In [16]:
results['heuristic'] = [generate_heuristic_knowledge(x) for x in huawei_dfs]

Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
Generating counted causality for Huawei log data: 100%|██████████| 100/100 [00:00<00:00, 240.31it/s]
Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
Generating counted causality for Huawei log data: 100%|██████████| 1000/1000 [00:01<00:00, 552.59it/s]
Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
Generating counted causality for Huawei log data: 100%|██████████| 10000/10000 [01:24<00:00, 118.05it/s]
Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
Generating counted causality for Huawei log data: 100%|██████████| 50000/50000 [16:13<00:00, 51.34it/s] 


### My method

In [18]:
def generate_my_method_knoweldge(huawei_df):
    vocab = set()
    for _, row in huawei_df.iterrows():
        for column in huawei_df.columns:
            name = column + "#" + str(row[column]).lower()
            if name not in vocab and row[column] != "":
                vocab.add(name)
    alg_df = pd.DataFrame()
    for column in vocab:
        attribute, value = tuple(column.split('#'))
        if column not in alg_df.columns:
            alg_df[column] = huawei_df[attribute].apply(lambda x: 1 if x == value else 0)
        else:
            alg_df[column] |= huawei_df[attribute].apply(lambda x: 1 if x == value else 0)
    obj = PC()
    output = obj.predict(alg_df)
    causality_records = []
    for edge in list(output.edges()):
        from_value, to_value = edge
        causality_records.append(
            {
                "parent_id": from_value,
                "parent_name": from_value.split("#")[1],
                "child_id": to_value,
                "child_name": to_value.split("#")[1],
            },
        )
    return pd.DataFrame.from_records(causality_records).drop_duplicates().reset_index(drop=True)

In [ ]:
results['my method'] = []
for huawei_df in huawei_dfs:
    start = time.time()
    results['my method'].append(generate_my_method_knoweldge(huawei_df))
    print(time.time() - start)